In [15]:
import pandas as pd
from pydataxm import *
import datetime as dt
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer, Date, Float  # Importar tipos necesarios
import  os
import psycopg2

In [16]:
DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)

In [17]:
#Definir periodo de los datos
today = dt.date.today()
month_ago = today - dt.timedelta(days=360) # Approximation of a month

In [18]:
#Conexion a la API
objetoAPI = pydataxm.ReadDB()

**TEMPERATURA SOLAR**

In [19]:
df_TemperaturaSolar = objetoAPI.request_data(
                                                "TempAmbSolar",           #Se indica el nombre de la métrica tal como se llama en el campo metricId
                                                "Recurso",               #Se indica el nombre de la entidad tal como se llama en el campo Entity
                                                month_ago,     #Corresponde a la fecha inicial de la consulta
                                                today)

df_TemperaturaSolar['Temperatura_solar']= df_temperatura_solar['Value']
#Renombrar
df_TemperaturaSolar.rename(columns={'Value': 'temperatura_solar',
                               'Code':'pkRecurso'
                              }, inplace=True)

df_TemperaturaSolar=df_TemperaturaSolar[['pkRecurso','Date','temperatura_solar']]

df_TemperaturaSolar['fecha_actualizacion'] = today
df_TemperaturaSolar.head()# Assume df_filtered is already defined from the preceding code



,pkRecurso,Date,temperatura_solar,fecha_actualizacion
0,2ZP9,2024-06-07,25.60926,2025-06-02
1,3ACC,2024-06-07,28.14450,2025-06-02
2,3B4L,2024-06-07,24.42759,2025-06-02
3,3EBN,2024-06-07,31.45000,2025-06-02
4,3EV1,2024-06-07,33.20000,2025-06-02


In [20]:
# Insertar datos en PostgreSQL (modo 'append' para no borrar datos existentes)
with engine.begin() as conn:  # <- Se hace COMMIT automático al final
    df_TemperaturaSolar.to_sql(
    name='temperatura_solar',          # Nombre de la tabla (sin esquema aquí)
    con=engine,
    schema='tablasxm_dev',       # Esquema especificado aquí
    if_exists='replace',          # Opción: 'append', 'replace' o 'fail'
    index=True,                 # Incluir el índice del DataFrame
    dtype={
           'pkrecurso': String(),     
           'date': Date(),
           'temperatura_solar': Float(),              
           'fecha_actualizacion': Date()
    })



print("Datos insertados correctamente.")

Datos insertados correctamente.


In [21]:
print(len(df_TemperaturaSolar))

8405
